In [6]:
import ftplib
import os
import xarray as xr
import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from scipy.interpolate import griddata
import subprocess
from google.cloud import storage
import ee
from rasterio.transform import from_origin

**Models**
* ACCESS1-0
* BCC-CSM1-1
* BCC-CSM1-1-M
* BNU-ESM
* CanESM2
* CCSM4
* CESM1-BGC
* CESM1-CAM5
* CMCC-CM
* CNRM-CM5
* CSIRO-Mk3-6-0
* FGOALS-G2
* FIO-ESM
* GFDL-CM3
* GFDL-ESM2G
* GFDL-ESM2M
* GISS-E2-H-CC
* GISS-E2-R
* GISS-E2-R-CC
* HadGEM2-AO
* HadGEM2-CC
* HadGEM2-ES
* INMCM4
* IPSL-CM5A-MR
* IPSL-CM5B-LR
* MIROC-ESM
* MIROC-ESM-CHEM
* MIROC5
* MPI-ESM-LR
* MPI-ESM-MR
* MRI-CGCM3
* NorESM1-M

**Variables**
- albedo
- baseflow
- del_SWE
- ET
- latent
- longwave_net
- pet_natveg
- precip
- Qair
- rainfall
- rel_Humid
- runoff
- sensible
- shortwave_in
- shortwave_net
- snow_melt
- snowfall
- soilMoist1
- soilMoist2
- soilMoist3
- sublimation_net
- SWE
- Tair
- Wdew
- windspeed

def download_file_from_ftp(ftp_path, local_destination=None):
    """
    Download a specific file from an FTP server.

    Parameters
    ----------
    ftp_path: str
        Full FTP path to the file (e.g., "ftp://server/path/to/file").
    local_destination: str
        Destination folder to save the file locally. Defaults to the current working directory.

    Returns
    -------
    str
        Full path to the downloaded file on the local machine.
    """
    # Parse the FTP server and file path
    ftp_server = ftp_path.split('/')[2]  # Extract server from URL
    file_path = '/'.join(ftp_path.split('/')[3:])  # Extract file path on server
    file_name = os.path.basename(file_path)  # Extract file name
    
    # Set the local destination
    if local_destination is None:
        local_destination = os.getcwd()
    if not os.path.exists(local_destination):
        os.makedirs(local_destination)
    
    local_file_path = os.path.join(local_destination, file_name)

    # Connect to FTP and download file
    ftp = ftplib.FTP(ftp_server)
    ftp.login()
    print(f"Connected to {ftp_server}. Downloading {file_name}...")
    
    with open(local_file_path, 'wb') as local_file:
        ftp.retrbinary(f"RETR {file_path}", local_file.write)
    
    ftp.quit()
    print(f"Download complete. File saved to: {local_file_path}")
    
    return local_file_path

    
ftp_url = "ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/loca_hydro/LOCA_VIC_dpierce_2017-02-28/CanESM2/vic_output.historical.netcdf/ET.1990.v0.nc"
local_folder = "./downloaded_files"  # Adjust as needed
download_file_from_ftp(ftp_url, local_folder)

In [5]:
import ftplib
import os

def download_model_files(model, year, variable, scenario="rcp85", local_destination=None):
    """
    Download a specific NetCDF file for a model and year from the FTP server.

    Parameters
    ----------
    model: str
        Model name (e.g., "CESM1-BGC", "CanESM2").
    year: int
        Year of the desired file.
    variable: str
        Variable to download (e.g., "ET").
    scenario: str
        Climate scenario for future projections (default: "rcp85"). "rcp45"
        Used for years after 2005.
    local_destination: str
        Destination folder to save files locally. Defaults to current working directory.

    Returns
    -------
    local_file_path: str
        Path to the downloaded file or None if the file was not found.
    """
    ftp_server = "gdo-dcp.ucllnl.org"
    base_path = "/pub/dcp/archive/cmip5/loca_hydro/LOCA_VIC_dpierce_2017-02-28"

    # Set local destination
    if local_destination is None:
        local_destination = os.getcwd()
    if not os.path.exists(local_destination):
        os.makedirs(local_destination)

    # Connect to FTP server
    ftp = ftplib.FTP(ftp_server)
    ftp.login()
    print(f"Connected to {ftp_server}.")

    # Determine folder based on year
    if year <= 2005:
        folder_path = f"{base_path}/{model}/vic_output.historical.netcdf"
    else:
        folder_path = f"{base_path}/{model}/vic_output.{scenario}.netcdf"

    # File name for the specific year
    file_name = f"{variable}.{year}.v0.nc"
    remote_file_path = f"{folder_path}/{file_name}"

    # Local file path
    local_file_path = os.path.join(local_destination, file_name)

    try:
        # Attempt to download the file
        print(f"Downloading {file_name} from {remote_file_path}...")
        with open(local_file_path, 'wb') as local_file:
            ftp.retrbinary(f"RETR {remote_file_path}", local_file.write)
        print(f"Successfully downloaded {file_name}.")
    except ftplib.error_perm as e:
        print(f"Error: File {file_name} not found on the server. ({e})")
        local_file_path = None
    finally:
        # Close the FTP connection
        ftp.quit()
        print("FTP connection closed.")

    return local_file_path


In [ ]:
""" model_name = "CanESM2"  # Adjust as needed
start = 2005
end = 2006
variable= 'ET'
#output_folder = "./downloaded_files"  # Specify your output folder

download_model_files(model_name, start, end,variable) """

Connected to gdo-dcp.ucllnl.org.
Successfully downloaded ET.2006.v0.nc.
Download process complete.


In [ ]:
""" # Open the .nc file
file_path = "./downloaded_files/ET.1990.v0.nc"  # Adjust to your file path
ds = xr.open_dataset(file_path)

# Inspect the dataset
print(ds)

# Select the variable "ET"
et = ds[variable]
 """

In [4]:

# Open the NetCDF file with rasterio
nc_path = r"C:\Users\bsf31\Documents\data\fire_probabilities\ET.2005.v0.nc"

with rasterio.open(f"NETCDF:{nc_path}") as src:
    print("Profile:", src.profile)  # Metadata similar to a .tif
    print("CRS:", src.crs)  # Coordinate reference system
    print("Transform:", src.transform)  # Affine transform (origin, resolution, rotation)
    print("Width, Height:", src.width, src.height)  # Number of columns and rows
    print("Bounds:", src.bounds)  # Extent (minx, miny, maxx, maxy)
    print("Resolution:", src.res)  # Cell size


Profile: {'driver': 'netCDF', 'dtype': 'float32', 'nodata': 1.0000000150474662e+30, 'width': 960, 'height': 490, 'count': 365, 'crs': None, 'transform': Affine(0.0625, 0.0, -126.0,
       0.0, -0.0625, 54.0), 'blockxsize': 960, 'blockysize': 490, 'tiled': False}
CRS: None
Transform: | 0.06, 0.00,-126.00|
| 0.00,-0.06, 54.00|
| 0.00, 0.00, 1.00|
Width, Height: 960 490
Bounds: BoundingBox(left=-126.0, bottom=23.375, right=-66.0, top=54.0)
Resolution: (0.0625, 0.0625)


In [ ]:
""" # Resample daily data to monthly averages
monthly_avg = et.resample(Time="1ME").mean(dim="Time")

monthly_range = monthly_avg.max(dim="Time") - monthly_avg.min(dim="Time")  # Shape: (Lat, Lon)
lat = ds["Lat"].values
lon = ds["Lon"].values
# Handle 1D or 2D lat/lon arrays
if len(lat.shape) == 1 and len(lon.shape) == 1:
    # 1D lat and lon
    lon2d, lat2d = np.meshgrid(lon, lat)
elif len(lat.shape) == 2 and len(lon.shape) == 2:
    # 2D lat and lon
    lat2d, lon2d = lat, lon
else:
    raise ValueError("Lat and Lon dimensions are incompatible. Check their shapes.")

# Create the interpolation grid
lon_interp = np.linspace(lon2d.min(), lon2d.max(), monthly_range.shape[1])
lat_interp = np.linspace(lat2d.min(), lat2d.max(), monthly_range.shape[0])
lon_mesh, lat_mesh = np.meshgrid(lon_interp, lat_interp)

# Flatten the original lat/lon and data values for interpolation
lat_lon_points = np.array([lat2d.ravel(), lon2d.ravel()]).T
monthly_range_flat = monthly_range.values.ravel()

# Perform interpolation to the regular grid
grid_data = griddata(lat_lon_points, monthly_range_flat, (lat_mesh, lon_mesh), method="linear")

# Flip the grid vertically for correct orientation
grid_data_flipped = np.flipud(grid_data)

nodata_value = -9999
# Replace NaN values in the grid with the NoData value
grid_data_flipped = np.where(np.isnan(grid_data_flipped), nodata_value, grid_data_flipped)

# Save as GeoTIFF
transform = from_bounds(
    lon_interp.min(), lat_interp.min(),
    lon_interp.max(), lat_interp.max(),
    grid_data_flipped.shape[1], grid_data_flipped.shape[0]
)

output_tif = "./ET_monthly_range.tif"
# Define the NoData value


with rasterio.open(
    output_tif,
    "w",
    driver="GTiff",
    height=grid_data_flipped.shape[0],
    width=grid_data_flipped.shape[1],
    count=1,
    dtype=grid_data_flipped.dtype,
    crs="EPSG:4326",
    transform=transform,
    nodata=nodata_value
) as dst:
    dst.write(grid_data_flipped, 1)

print(f"GeoTIFF saved to {output_tif}")
 """


GeoTIFF saved to ./ET_monthly_range.tif


In [ ]:
""" # Configuration
model_name = "CanESM2"
start_year = 2005
end_year = 2006
variable = "ET"
output_folder = "./output_tifs"
os.makedirs(output_folder, exist_ok=True)

# Loop through years
for year in range(start_year, end_year + 1):
    # Step 1: Download NetCDF file
    file_path = download_model_files(model_name, year, variable)
    
    # Step 2: Open the NetCDF file
    ds = xr.open_dataset(file_path)
    et = ds[variable]

    # Step 3: Resample daily data to monthly averages
    monthly_avg = et.resample(Time="1ME").mean(dim="Time")

    # Step 4: Calculate the range of monthly averages
    monthly_range = monthly_avg.max(dim="Time") - monthly_avg.min(dim="Time")  # Shape: (Lat, Lon)

    # Step 5: Extract lat/lon and handle 1D or 2D arrays
    lat = ds["Lat"].values
    lon = ds["Lon"].values

    if len(lat.shape) == 1 and len(lon.shape) == 1:
        lon2d, lat2d = np.meshgrid(lon, lat)
    elif len(lat.shape) == 2 and len(lon.shape) == 2:
        lat2d, lon2d = lat, lon
    else:
        raise ValueError("Lat and Lon dimensions are incompatible. Check their shapes.")

    # Step 6: Create the interpolation grid
    lon_interp = np.linspace(lon2d.min(), lon2d.max(), monthly_range.shape[1])
    lat_interp = np.linspace(lat2d.min(), lat2d.max(), monthly_range.shape[0])
    lon_mesh, lat_mesh = np.meshgrid(lon_interp, lat_interp)

    # Step 7: Interpolate the data to a regular grid
    lat_lon_points = np.array([lat2d.ravel(), lon2d.ravel()]).T
    monthly_range_flat = monthly_range.values.ravel()
    grid_data = griddata(lat_lon_points, monthly_range_flat, (lat_mesh, lon_mesh), method="linear")

    # Flip the grid vertically for correct orientation
    grid_data_flipped = np.flipud(grid_data)

    # Step 8: Replace NaN values with NoData value
    nodata_value = -9999
    grid_data_flipped = np.where(np.isnan(grid_data_flipped), nodata_value, grid_data_flipped)

    # Step 9: Save the result as a GeoTIFF
    transform = from_bounds(
        lon_interp.min(), lat_interp.min(),
        lon_interp.max(), lat_interp.max(),
        grid_data_flipped.shape[1], grid_data_flipped.shape[0]
    )

    output_tif = os.path.join(output_folder, f"{variable}_monthly_range_{year}.tif")
    with rasterio.open(
        output_tif,
        "w",
        driver="GTiff",
        height=grid_data_flipped.shape[0],
        width=grid_data_flipped.shape[1],
        count=1,
        dtype=grid_data_flipped.dtype,
        crs="EPSG:4326",
        transform=transform,
        nodata=nodata_value
    ) as dst:
        dst.write(grid_data_flipped, 1)

    print(f"GeoTIFF saved to {output_tif}") """

Connected to gdo-dcp.ucllnl.org.
Successfully downloaded ET.2005.v0.nc.
FTP connection closed.
GeoTIFF saved to ./output_tifs\ET_monthly_range_2005.tif
Connected to gdo-dcp.ucllnl.org.
Successfully downloaded ET.2006.v0.nc.
FTP connection closed.
GeoTIFF saved to ./output_tifs\ET_monthly_range_2006.tif


In [ ]:
def process_year(model, year, variable, scenario, output_folder, gcp_bucket, ee_path):
    # Step 1: Download .nc File
    nc_file_path = download_model_files(model, year, variable, scenario, output_folder)

    if not nc_file_path:
        print(f"No .nc file for {year}. Skipping...")
        return

    # Step 2: Convert .nc to .tif
    ds = xr.open_dataset(nc_file_path)
    et = ds[variable]
    monthly_avg = et.resample(Time="1MS").mean(dim="Time")
    monthly_range = monthly_avg.max(dim="Time") - monthly_avg.min(dim="Time")
    
    # Handle lat/lon
    lat, lon = ds["Lat"].values, ds["Lon"].values
    lon2d, lat2d = np.meshgrid(lon, lat)
    lon_interp = np.linspace(lon2d.min(), lon2d.max(), monthly_range.shape[1])
    lat_interp = np.linspace(lat2d.min(), lat2d.max(), monthly_range.shape[0])
    lon_mesh, lat_mesh = np.meshgrid(lon_interp, lat_interp)
    lat_lon_points = np.array([lat2d.ravel(), lon2d.ravel()]).T
    monthly_range_flat = monthly_range.values.ravel()
    grid_data = griddata(lat_lon_points, monthly_range_flat, (lat_mesh, lon_mesh), method="linear")
    grid_data_flipped = np.flipud(np.where(np.isnan(grid_data), -9999, grid_data))

    transform = from_bounds(
        lon_interp.min(), lat_interp.min(), lon_interp.max(), lat_interp.max(),
        grid_data_flipped.shape[1], grid_data_flipped.shape[0]
    )
    tif_file = os.path.join(output_folder, f"{variable}_monthly_range_{year}.tif")
    with rasterio.open(
        tif_file, "w", driver="GTiff", height=grid_data_flipped.shape[0],
        width=grid_data_flipped.shape[1], count=1, dtype=grid_data_flipped.dtype,
        crs="EPSG:4326", transform=transform, nodata=-9999
    ) as dst:
        dst.write(grid_data_flipped, 1)
    print(f"GeoTIFF saved to {tif_file}")

    # Delete .nc File
    os.remove(nc_file_path)
    print(f"Deleted .nc file: {nc_file_path}")

    # Step 3: Save as COG
    cog_file = tif_file.replace(".tif", "_cog.tif")
    cog_cmd = f'gdal_translate {tif_file} {cog_file} -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW'
    subprocess.run(cog_cmd, shell=True)
    os.remove(tif_file)
    print(f"COG created and original GeoTIFF deleted: {cog_file}")

    # Step 4: Upload to GCP
    cp_cmd = f"gsutil cp {cog_file} gs://{gcp_bucket}/{os.path.basename(cog_file)}"
    subprocess.run(cp_cmd, shell=True)
    print(f"Uploaded COG to GCP: {cog_file}")

    # Step 5: Register in Earth Engine
    ee_image = ee.Image.loadGeoTIFF(f"gs://{gcp_bucket}/{os.path.basename(cog_file)}")
    ee_image = ee_image.set(
        "system:time_start", ee.Date(f"{year}-01-01").millis(),
        "system:time_end", ee.Date(f"{year}-12-31").millis(),
        "model", model,
        "variable", variable,
        "scenario", scenario
    )
    asset_id = f"projects/{PROJECT_ID}/assets/{ee_path}/{variable}_monthly_range_{year}"
    ee.data.createAsset({"type": "IMAGE_COLLECTION"}, ee_path, allowExisting=True)
    ee.batch.Export.image.toAsset(
        image=ee_image,
        description=f"{variable}_{year}",
        assetId=asset_id,
        region=ee_image.geometry().bounds(),
        scale=5000,
        maxPixels=1e13,
    ).start()
    print(f"Registered in Earth Engine: {asset_id}")


In [ ]:
# Configuration
output_folder = "./output"
gcp_bucket = "my-gcp-bucket"
ee_path = "my-ee-collection-path"
start_year, end_year = 2005, 2006
model = "CanESM2"
variable = "ET"
scenario = "rcp85"

# Initialize Earth Engine
ee.Initialize()

# Process each year
for year in range(start_year, end_year + 1):
    process_year(model, year, variable, scenario, output_folder, gcp_bucket, ee_path)


In [ ]:
# Configuration
output_folder = "./output"
gcp_bucket = "my-gcp-bucket"
ee_path = "my-ee-collection-path"
start_year, end_year = 2005, 2006
model = "CanESM2"
variable = "ET"
scenario = "rcp85"

In [16]:
# Step 1: Download .nc File
nc_file_path = r"C:\Users\bsf31\Documents\data\fire_probabilities\ET.2005.v0.nc"


# Step 2: Open NetCDF and extract variable
ds = xr.open_dataset(nc_file_path)
et = ds["ET"]

# Compute monthly statistics
monthly_avg = et.resample(Time="1MS").mean(dim="Time")
monthly_range = monthly_avg.max(dim="Time") - monthly_avg.min(dim="Time")

# Get spatial dimensions
lat = ds["Lat"].values  # Latitude
lon = ds["Lon"].values  # Longitude
data_array = monthly_range.values  # Extract numpy array


data_array = np.flipud(data_array)
lat = lat[::-1]  # Reverse to match flipped array

# Get resolution (Check if lat/lon are regularly spaced)
res_x = abs(lon[1] - lon[0])
res_y = abs(lat[1] - lat[0])

# Adjust bounds to account for pixel edges (NetCDF may store cell centers)
lat_min, lat_max = lat.min() - (res_y / 2), lat.max() + (res_y / 2)
lon_min, lon_max = lon.min() - (res_x / 2), lon.max() + (res_x / 2)

# Define transformation using corrected bounds
transform = from_bounds(lon_min, lat_min, lon_max, lat_max, data_array.shape[1], data_array.shape[0])

# Check for CRS in NetCDF (If available, use it instead of EPSG:4326)
if "spatial_ref" in ds.variables:
    crs = ds.spatial_ref.attrs.get("crs", "EPSG:4326")  # Get CRS if present
else:
    crs = "EPSG:4326"  # Default assumption

# Define output GeoTIFF file path
tif_file = os.path.join(r'C:\Users\bsf31\Documents\data\fire_probabilities', f"ET_monthly_range_2005.tif")

# Create GeoTIFF using rasterio
with rasterio.open(
    tif_file, "w", driver="GTiff",
    height=data_array.shape[0], width=data_array.shape[1],
    count=1, dtype=data_array.dtype,
    crs=crs, transform=transform, nodata=-9999
) as dst:
    dst.write(data_array, 1)

print(f"GeoTIFF saved to {tif_file}")

# Close dataset
ds.close()

GeoTIFF saved to C:\Users\bsf31\Documents\data\fire_probabilities\ET_monthly_range_2005.tif
